In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Osteoarthritis"
cohort = "GSE56409"

# Input paths
in_trait_dir = "../../input/GEO/Osteoarthritis"
in_cohort_dir = "../../input/GEO/Osteoarthritis/GSE56409"

# Output paths
out_data_file = "../../output/preprocess/Osteoarthritis/GSE56409.csv"
out_gene_data_file = "../../output/preprocess/Osteoarthritis/gene_data/GSE56409.csv"
out_clinical_data_file = "../../output/preprocess/Osteoarthritis/clinical_data/GSE56409.csv"
json_path = "../../output/preprocess/Osteoarthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Stromal transcriptional profiles reveal hierarchies of anatomical site, serum response and disease and identify disease specific pathways"
!Series_summary	"Synovial fibroblasts in persistent inflammatory arthritis have been suggested to have parallels with cancer growth and wound healing, both of which involve a stereotypical serum response program. We tested the hypothesis that a serum response program can be used to classify diseased tissues, and investigated the serum response program in fibroblasts from multiple anatomical sites and two diseases. To test our hypothesis we utilized a bioinformatics approach to explore a publicly available microarray dataset including RA, OA and normal synovial tissue, then extended those findings in a new microarray dataset representing matched synovial, bone marrow and skin fibroblasts cultured from RA and OA patients undergoing arthroplasty. The classical fibroblast serum response program discretely classifie

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background info, this is a microarray dataset comparing synovial fibroblasts
# from RA and OA patients, so it likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# From the Sample Characteristics Dictionary, we can identify:

# 2.1 Data Availability
# Analyzing the Sample Characteristics Dictionary:
# Key 1 has 'disease: RA', 'disease: OA' which relates to our trait (Osteoarthritis)
trait_row = 1  

# There's no age information in the sample characteristics 
age_row = None

# There's no gender information in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(val):
    """Convert trait values to binary (0=no OA, 1=OA)"""
    if val is None:
        return None
    # Split by colon and get the value part
    if ":" in val:
        val = val.split(":", 1)[1].strip()
    
    # For Osteoarthritis
    if val.upper() == "OA":
        return 1
    # For Rheumatoid Arthritis
    elif val.upper() == "RA":
        return 0
    else:
        return None

def convert_age(val):
    """Convert age values to continuous numbers"""
    # Not needed since age data is not available
    return None

def convert_gender(val):
    """Convert gender values to binary (0=female, 1=male)"""
    # Not needed since gender data is not available
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Validate and save the initial cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we extract clinical features
if trait_row is not None:
    # Create a DataFrame that simulates the expected structure for geo_select_clinical_features
    # We need rows representing feature types and columns representing samples
    
    # Create a DataFrame with categorical traits from sample characteristics
    # First, prepare columns with sample names (we'll create simplified sample IDs)
    sample_names = ['Sample_' + str(i+1) for i in range(4)]  # Create 4 samples for demonstration
    
    # Create a DataFrame where rows are the characteristics and columns are samples
    data = {
        sample_names[0]: ['disease: OA', None, None],
        sample_names[1]: ['disease: OA', None, None],
        sample_names[2]: ['disease: RA', None, None],
        sample_names[3]: ['disease: RA', None, None]
    }
    
    # Create the DataFrame with the correct structure
    # Index will be the row numbers that correspond to trait_row, age_row, gender_row
    clinical_data = pd.DataFrame(data, index=[0, 1, 2])
    
    # Extract clinical features using the function from the library
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Clinical Features Preview:")
    print(preview)
    
    # Save the clinical features to the output file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Features Preview:
{'Sample_1': [nan], 'Sample_2': [nan], 'Sample_3': [nan], 'Sample_4': [nan]}
Clinical data saved to ../../output/preprocess/Osteoarthritis/clinical_data/GSE56409.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Osteoarthritis/GSE56409/GSE56409_series_matrix.txt.gz


Gene data shape: (21742, 102)
First 20 gene/probe identifiers:
Index(['1007_s_at', '1294_at', '1405_i_at', '1487_at', '1552257_a_at',
       '1552264_a_at', '1552269_at', '1552274_at', '1552275_s_at',
       '1552277_a_at', '1552286_at', '1552288_at', '1552289_a_at',
       '1552291_at', '1552293_at', '1552295_a_at', '1552299_at', '1552302_at',
       '1552306_at', '1552307_a_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers seen in the gene expression data look like probe IDs from an Affymetrix microarray,
# not standard human gene symbols. They follow the pattern of numerical identifiers with suffixes 
# like "_at", "_s_at", and "_a_at", which are characteristic of Affymetrix probe identifiers.
# These will need to be mapped to standard gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observe gene identifiers and determine mapping columns
# Based on the gene annotation preview, 'ID' column contains probe IDs that match the gene expression data's index,
# and 'Gene Symbol' column contains the gene symbols we need to map to.

# 2. Get a gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print(f"Gene mapping dataframe shape: {gene_mapping.shape}")
print("Preview of gene mapping dataframe:")
print(preview_df(gene_mapping, n=5))

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("Preview of gene expression data (first 5 genes, first 5 samples):")
# Display first 5 rows and first 5 columns
print(gene_data.iloc[:5, :5])

# Normalize gene symbols to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalizing gene symbols: {gene_data.shape}")

# Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (45782, 2)
Preview of gene mapping dataframe:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}


Gene expression data shape after mapping: (12392, 102)
Preview of gene expression data (first 5 genes, first 5 samples):
          GSM1360956  GSM1360957  GSM1360958  GSM1360959  GSM1360960
Gene                                                                
A1BG        5.274919    5.738628    3.336548    3.457008    4.083493
A1BG-AS1    4.453343    4.914406    3.572509    4.002934    3.573421
A2M         5.798740    6.459806    5.214447    5.554075    5.285597
A2M-AS1     4.810059    5.034640    4.747397    5.095043    5.651134
A4GALT      5.362888    5.379710    5.655486    5.657037    4.360879
Gene expression data shape after normalizing gene symbols: (12087, 102)


Gene expression data saved to ../../output/preprocess/Osteoarthritis/gene_data/GSE56409.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data 
# (This was already done in the previous step, so no need to repeat)
print(f"Normalized gene data shape: {gene_data.shape}")

# 2. Examine the sample IDs in the gene expression data to understand the structure
print("Gene data column names (sample IDs):")
print(gene_data.columns[:5])  # Print first 5 for brevity

# Inspect the clinical data format from the matrix file directly
background_info, clinical_data = get_background_and_clinical_data(matrix_file)
print("\nRaw clinical data structure:")
print(f"Clinical data shape: {clinical_data.shape}")
print(f"Clinical data columns: {clinical_data.columns[:5]}")

# Get the sample characteristics to re-extract the disease information
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print("\nSample characteristics dictionary:")
print(sample_characteristics_dict)

# 3. Directly create clinical features from the raw data again
# Verify trait row contains the disease information (OA vs RA)
print("\nValues in trait row:")
trait_values = clinical_data.iloc[trait_row].values
print(trait_values[:5])

# Create clinical dataframe with proper structure
# First get the sample IDs from gene data as these are our actual sample identifiers
sample_ids = gene_data.columns.tolist()

# Create the clinical features dataframe with those sample IDs
clinical_features = pd.DataFrame(index=[trait], columns=sample_ids)

# Fill the clinical features with our trait values by mapping GSM IDs to actual values
for col in clinical_data.columns:
    if col in sample_ids:
        # Extract the disease value and convert it
        disease_val = clinical_data.iloc[trait_row][col]
        clinical_features.loc[trait, col] = convert_trait(disease_val)

print("\nCreated clinical features dataframe:")
print(f"Shape: {clinical_features.shape}")
print(clinical_features.iloc[:, :5])  # Show first 5 columns

# 4. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
print(f"\nLinked data shape before handling missing values: {linked_data.shape}")

# 5. Handle missing values - we need to use the actual column name, not the trait variable
# First identify the actual trait column name in the linked data
trait_column = clinical_features.index[0]  # This should be 'Osteoarthritis'
print(f"Actual trait column in linked data: {trait_column}")

# Now handle missing values with the correct column name
linked_data_clean = handle_missing_values(linked_data, trait_column)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 6. Evaluate bias in trait and demographic features
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait_column)

# 7. Conduct final quality validation
note = "Dataset contains gene expression data from synovial fibroblasts of RA and OA patients. Data includes high serum and low serum responses."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=(linked_data_clean.shape[0] > 0),
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 8. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")

Normalized gene data shape: (12087, 102)
Gene data column names (sample IDs):
Index(['GSM1360956', 'GSM1360957', 'GSM1360958', 'GSM1360959', 'GSM1360960'], dtype='object')

Raw clinical data structure:
Clinical data shape: (3, 103)
Clinical data columns: Index(['!Sample_geo_accession', 'GSM1360956', 'GSM1360957', 'GSM1360958',
       'GSM1360959'],
      dtype='object')

Sample characteristics dictionary:
{0: ['tissue: Synovium', 'tissue: Skin', 'tissue: Bone Marrow'], 1: ['disease: RA', 'disease: OA'], 2: ['serum: Low Serum', 'serum: High Serum']}

Values in trait row:
['!Sample_characteristics_ch1' 'disease: RA' 'disease: RA' 'disease: RA'
 'disease: RA']

Created clinical features dataframe:
Shape: (1, 102)
               GSM1360956 GSM1360957 GSM1360958 GSM1360959 GSM1360960
Osteoarthritis          0          0          0          0          0

Linked data shape before handling missing values: (102, 12088)
Actual trait column in linked data: Osteoarthritis


/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())


Linked data shape after handling missing values: (102, 12088)
For the feature 'Osteoarthritis', the least common label is '1' with 34 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Osteoarthritis' in this dataset is fine.



Linked data saved to ../../output/preprocess/Osteoarthritis/GSE56409.csv
